In [1]:
import os

import pandas as pd
from langdetect import detect

import src.mongos as mg

%load_ext autoreload
%autoreload 2

# Const

In [2]:
db_name = 'youtube_comment'
# collection = 'comments'

vids = [  # vid(video_ids) goes to collection name
    'XsX3ATc3FbA',  # 작은 것들을 위한 시
    'rBG5L7UsUxA',  # 데뷔곡: 2 COOL 4 SKOOL
]

cids = [
    'comments',
    'comments_rBG5L7UsUxA',
]

# Add langs

In [3]:
from langdetect.lang_detect_exception import LangDetectException

error_codes = {
    0: 'NoTextError',
    1: 'FormatError',
    2: 'FileLoadError',
    3: 'DuplicateLangError',
    4: 'NeedLoadProfileError',
    5: 'CantDetectError',
    6: 'CantOpenTrainData',
    7: 'TrainDataFormatError',
    8: 'InitParamError',
}

def detect_lang(row):
    text = row['text']
    author = row['author']
    try:
        return detect(text)
    except (LangDetectException, TypeError) as e:
        try:
            return detect(author)
        except LangDetectException as e:
            return error_codes[e.code]
        except TypeError as e:
            return 'TypeError'

def detect_text(t):
    try:
        return detect(t)
    except LangDetectException as e:
        return error_codes[e.code]

def detect_author(a):
    try:
        return detect(a)
    except LangDetectException as e:
        return error_codes[e.code]
    

def get_langs(comments):
#     comments = raw_comments.copy()

#     comments['639-1'] = comments[['text', 'author']].apply(detect_lang, axis=1)
#     comments['639-1'] = comments['text'].apply(detect_text)
#     comments = comments.merge(langs[['ISO language name', '639-1']], on='639-1', how='left')
#     comments['ISO language name'].fillna(comments['639-1'], inplace=True)
#     return comments
    return comments['text'].apply(detect_text)


def detect_mp(row):
    t = row['text']
    try:
        return detect(t)
    except LangDetectException as e:
        return error_codes[e.code]
    except TypeError as e:
        return 'TypeError'


# comments['lang_text'] = comments['text'].apply(detect_text)
# comments['lang_author'] = comments['author'].apply(detect_author)
# comments = comments.merge(langs[['ISO language name', '639-1']], left_on='lang_text', right_on='639-1', how='left')

# comments = add_langs(raw_comments)

# print(len(comments))
# comments

# Multiprocessing

In [4]:
from multiprocessing import  Pool
from functools import partial
import numpy as np

def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(func, data_subset):
    return data_subset.apply(func, axis=1)

def parallelize_on_rows(data, func, num_of_processes=8):
    return parallelize(data, partial(run_on_subset, func), num_of_processes)

# Test MP

In [5]:
# comments = mg.read_mongo(db_name, collection, query={'langCode': {'$exists': False}}, limit=1000, no_id=False,)
# comments['langCode'] = parallelize_on_rows(comments, detect_mp, num_of_processes=4)
# comments.head()

# Execution

In [6]:
collection = cids[1]

## Before Update

In [7]:
comments = mg.read_mongo(db_name, collection, query={'langCode': {'$exists': True}}, no_id=False)
len(comments)

81000

## Load comments

In [8]:
while(True):
    comments = mg.read_mongo(db_name, collection, query={'langCode': {'$exists': False}}, limit=1000, no_id=False)
    if not len(comments):
        break

#     comments['langCode'] = get_langs(comments)  # %timeit : 9 seconds for 1000 rows
    comments['langCode'] = parallelize_on_rows(comments, detect_mp, num_of_processes=4)
    mg.update_mongo(comments, db_name, collection, ['_id'])

- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
match, insert, modify, upsert
1000 0 1000 0
- bulk_write result:
matc

## After Update

In [9]:
comments = mg.read_mongo(db_name, collection, query={'langCode': {'$exists': True}}, no_id=False)
len(comments)

120394